### <h1><center> To develop a range of data analytics specific preditive models to help automate the sourcing and uncovering of exceptional fintech start-ups in VC Investing.

## BRIEF BACKGROUND TO THE CHOSEN DATASET

Data is collected and curated from two separate sources, Viziologi and Crunchbase. Both sources are known aggregators of information of venture-backed firms and corresponding funding rounds.
 

*   From Viziologi, a dataset of 1485 instances and 19 variables is obtained. The dataset spans both active and inactive, private companies who have raised Pre-Seed, Seed, Series A, Series B and Series C rounds of funding, as well as public companies. The dataset contains 19 variables: company name, sector, industry, business model patterns, company description, company url, headquarters, area, country, founding year, company type, tags, social impact, life impact, emotional impact, functional impact, digital maturity, similar business patterns and similar companies.

*   The Viziologi dataset is supplemented with additional data from Crunchbase. A dataset of 1008 instances and 11 variables is scraped from Viziologi: name, operating_status, ipo, company type, latest funding type, latest_funding_date, second latest funding type, second latest funding date, patents, trademarks, hub.

## MERGING THE VIZIOLOGI AND CRUNCHBASE DATASETS

The Viziologi and Crunchbase datasets are first cleaned and pre-processed, before being merged into one dataset (df_merged.csv). Companies that are represented in both the Viziologi and Crunchbase datasets are included in the merged dataset. Companies that fail to fulfil the recency requirement – founded before 2007 (in accordance with the recency requirement of the RFM model), are dropped. In total, the merged dataset comprises of 526 recent start-ups.

We shall use the merged dataset (df_merged) to perform data cleaning and preparation, feature engineering, exploratory data analysis (EDA), machine learning modelling, as well as  evaluation, in a bid to identify and source exceptional fintech startups for VC investing. 

## THE CHOSEN MACHINE LEARNING PREDICTIVE MODELS

We shall build and evaluate the performance of at least five machine learning classification models, i.e.:

1. Logistics Regression
2. Naive Bayes Classification
3. Support Vector Machine (SVM)
4. Kth Nearest Neighbour
5. TTree-Based Methods

## IMPORTING NECESSARY LIBRARIES AND MODULES 

In [1]:
# import the following libraries and modules that are required for use in this notebook

import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from scipy import stats

## LOADING THE VIZIOLOGI AND CRUNCHBASE DATASETS ONTO THE NOTEBOOK

In [6]:
# load the viziologi dataset and view the first 3 rows
df_viziologi=pd.read_csv('/content/Viziologi.csv')
df_viziologi.head(3)

,name,sector,category,business_patterns,description,url,hq,area,country,founded,company_type,tags,social_impact,life_impact,emotional_impact,functional_impact,digital_maturity,similar_business_patterns,similar_companies
0,Argo AI,Transportation,Automotive,"Transportation as a Service (TaaS), Subscripti...","Argo AI is an artificial intelligence company,...",https://www.argo.ai/,Pittsburgh,Pennsylvania,US,2017,Private,"Artificial Intelligence, autonomous vehicles, ...",NaN,NaN,"provides access, design/aesthetics, attractive...","saves time, simplifies, reduces risk, organize...",Digirati,Transportation as a Service (TaaS)\nSubscripti...,['Cruise Automation\nArgo AI\nZoox\nNutonomy\n...
1,Call 9,Healthcare,Professional Services,"Healthcare, Subscription, Pay as you go, On-de...",Call9. Inc. develops a mobile technology platf...,https://www.call9.com,Brooklyn,New York,US,2017,Private,"medical care, health care, information technol...",NaN,NaN,"provides access, reduces anxiety","saves time, simplifies, reduces cost, connects...",Digirati,Healthcare\nSubscription\nPay as you go\nOn-de...,['StarRez\nVettery\nHandy\nZilok\nRinse\nRoom2...
2,DeepAI,Technology,Software,"Open-source, Community-funded, Pay as you go, ...",DeepAI is driven to push the boundaries of art...,https://deepai.org/,Santa Barbara,California,US,2017,Private,"Machine Learning, ML, innovation, Artificial I...",NaN,"self-actualization, affiliation/belonging",provides access,"simplifies, saves time, reduces effort, avoids...",Digirati,Open-source\nCommunity-funded\nPay as you go\n...,['Clarifai\nVize AI\nProphesee\nDeep Cognition...


In [7]:
# work out the shape of the viziologi dataframe
df_viziologi.shape

(1485, 19)

As can be observed, the viziologi dataframe consists of 1485 rows and 19 columns following loading the dataset into the notebook.

In [4]:
# load the crunchbase dataset and view the first 3 rows
df_crunchbase=pd.read_csv('/content/Crunchbase.csv')
df_crunchbase.head(3)

,name,operating_status,ipo,company_type,latest_funding_type,latest_funding_date,second_latest_funding_type,second_latest_funding_date,patents,trademarks,hub
0,Argo AI,Active,Private,For Profit,Corporate Round - Argo AI,"Jul 12, 2019",Funding Round - Argo AI,"Feb 13, 2017",46,7,NaN
1,DeepAI,Seed,Private,NaN,Seed Round - DeepAI,"Mar 22, 2019",NaN,NaN,NaN,NaN,NaN
2,Growsel,NaN,Private,Non-profit,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# work out the shape of the crunchbase dataframe
df_crunchbase.shape

(1008, 11)

As can be observed, the crunchbase dataframe consists of 1008 rows and 11 columns following loading this dataset into the notebook.

## DATA CLEANING AND PRE-PROCESSING OF THE VIZIOLOGI DATASET

#### The process is initiated by checking for the presence of missing values in the viziologi dataframe.

As can be observed from the results below, three features in the viziologi dataset consist of missing values. Features 'company_type', 'social_impact', and 'life_impact' each consist of 22, 1376, and 314 missing values respectively.

In [10]:
# checking for presence of any missing values in the viziologi dataframe 
df_viziologi.isnull().sum()

name                            0
sector                          0
category                        0
business_patterns               0
description                     0
url                             0
hq                              0
area                            0
country                         0
founded                         0
company_type                   22
tags                            0
social_impact                1376
life_impact                   314
emotional_impact                4
functional_impact               0
digital_maturity                0
similar_business_patterns       0
similar_companies               0
dtype: int64

#### The next step inculcates eliminating all unnecessary columns from the viziologi dataframe.

Features "url","digital_maturity","similar_business_patterns","similar_companies","company_type","social_impact", "life_impact","emotional_impact","functional_impact" will be dropped as they do not have significant impact on the suggested classification problem. This step includes the elimination of the three features with missing or null values; i.e. "company_type", "social_impact", as well as "life_impact".

In [11]:
# drop the following columns from the viziologi dataframe 
df_viziologi.drop(columns=["url","digital_maturity","similar_business_patterns","similar_companies","company_type","social_impact",
                       "life_impact","emotional_impact","functional_impact"],inplace=True)


#### This steps involves inspecting the shape of the viziologi dataframe following removal of the aforementioned features.

As can be seen from the results given below, the viziologi dataframe now consists of 10 columns, from an initially stated 19 columns. 9 columns have been successfully dropped.

In [12]:
# work out the shape of the viziologi dataframe again
df_viziologi.shape

(1485, 10)

#### Inspect for the presence of missing or null values in the viziologi dataframe.

As can be observed from the results below, the dataframe now does not contain any missing values.

In [13]:
# checking for presence of any missing values in the viziologi dataframe, for the second time
df_viziologi.isnull().sum()

name                 0
sector               0
category             0
business_patterns    0
description          0
hq                   0
area                 0
country              0
founded              0
tags                 0
dtype: int64

## DATA CLEANING AND PRE-PROCESSING OF THE CRUNCHBASE DATASET

#### Cleaning of the Crunchbase dataset is initiated by checking for the presence of missing values.

As can be observed from the results given below, all columns in this dataset consist of missing values.

In [14]:
df_crunchbase.isnull().sum()

name                           37
operating_status              493
ipo                            46
company_type                  496
latest_funding_type           398
latest_funding_date           398
second_latest_funding_type    499
second_latest_funding_date    499
patents                       343
trademarks                    590
hub                           937
dtype: int64

We shall conduct some in-depth feature engineering of these columns, which will, in turn, help in removing these missing values.

## FEATURE ENGINEERING VARIABLES IN THE CRUNCHBASE DATAFRAME

#### All non-profit companies are eliminated from the dataframe.

Since we are only interested in startups that are going to generate profit and maximize the return on investment (ROI) for their VC investors, we shall drop all the non-profit companies and only retain the 'for-profit' companies.

In [16]:
# dropping all non-profit companies from the crunchbase dataframe
df_crunchbase.drop(df_crunchbase[df_crunchbase["company_type"]=="Non-profit"].index, inplace=True)


#### The latest investment funding types for each company are extracted.

In this section, we extract the latest investment funding types for each company. This information is significant as we shall use it in the classification machine learning problem, in order to help identify if companies are successful based on the type of investment funds they have received so far.

In [21]:
# Extract the substrings before the '-' delimiter from the 'latest funding type' column and add them to a new column  
latest_funding_type_list = []
for i in df_crunchbase['latest_funding_type'].iloc[:]:
  sentence_with_hyphen = str(i)
  latest_investment_fund_type = sentence_with_hyphen.split("-",1)[0]
  latest_funding_type_list.append(latest_investment_fund_type)

# add the extracted substrings into a new list
df_crunchbase["latest_funding_type_column"] = latest_funding_type_list

# view the first 3 rows of the new crunchbase dataframe
df_crunchbase.head(3)

,name,operating_status,ipo,company_type,latest_funding_type,latest_funding_date,second_latest_funding_type,second_latest_funding_date,patents,trademarks,hub,latest_funding_type_column
0,Argo AI,Active,Private,For Profit,Corporate Round - Argo AI,"Jul 12, 2019",Funding Round - Argo AI,"Feb 13, 2017",46,7,NaN,Corporate Round
1,DeepAI,Seed,Private,NaN,Seed Round - DeepAI,"Mar 22, 2019",NaN,NaN,NaN,NaN,NaN,Seed Round
3,Juni Learning,Early Stage Venture,Private,For Profit,Series A - Juni Learning,"Mar 12, 2020",Seed Round - Juni Learning,"May 31, 2019",NaN,NaN,NaN,Series A


#### The latest investment funding types (extracted from latest_funding_type_column) are inspected.

The results are displayed below.

In [22]:
# printing all the latest investment funding types for each company
df_crunchbase["latest_funding_type_column"].value_counts()

nan                       395
Venture Round             103
Post                       75
Series B                   59
Series C                   43
Seed Round                 42
Series A                   41
Series D                   32
Series E                   31
Private Equity Round       27
Secondary Market           26
Corporate Round            23
Debt Financing             19
Grant                      18
Funding Round              17
Series F                   14
Series G                   10
Initial Coin Offering       5
Angel Round                 4
Convertible Note            4
Equity Crowdfunding         3
Series H                    2
Series E                    1
Series I                    1
Non Equity Assistance       1
Series J                    1
Product Crowdfunding        1
Series C                    1
Name: latest_funding_type_column, dtype: int64

#### Next, the second latest investment funding types for each company are extracted.

In [23]:
# Extract the substrings before the '-' delimiter from the 'second latest funding type' column and add them to a new column  
second_latest_funding_type_list = []
for i in df_crunchbase['second_latest_funding_type'].iloc[:]:
  sentence_with_hyphen_1 = str(i)
  second_latest_investment_fund_type = sentence_with_hyphen_1.split("-",1)[0]
  second_latest_funding_type_list.append(second_latest_investment_fund_type)

# add the extracted substrings into a new list
df_crunchbase["second_latest_funding_type_column"] = second_latest_funding_type_list

# view the first 3 rows of the new crunchbase dataframe
df_crunchbase.head(3)

,name,operating_status,ipo,company_type,latest_funding_type,latest_funding_date,second_latest_funding_type,second_latest_funding_date,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column
0,Argo AI,Active,Private,For Profit,Corporate Round - Argo AI,"Jul 12, 2019",Funding Round - Argo AI,"Feb 13, 2017",46,7,NaN,Corporate Round,Funding Round
1,DeepAI,Seed,Private,NaN,Seed Round - DeepAI,"Mar 22, 2019",NaN,NaN,NaN,NaN,NaN,Seed Round,nan
3,Juni Learning,Early Stage Venture,Private,For Profit,Series A - Juni Learning,"Mar 12, 2020",Seed Round - Juni Learning,"May 31, 2019",NaN,NaN,NaN,Series A,Seed Round


#### The second latest investment funding types that have been extracted from 'second_latest_funding_type_column' are inspected.

The results are displayed below.

In [24]:
# printing all the second latest investment funding types for each company 
df_crunchbase["second_latest_funding_type_column"].value_counts()

nan                       496
Series A                   63
Venture Round              62
Series B                   52
Seed Round                 50
Series D                   43
Series C                   35
Series E                   33
Post                       29
Secondary Market           23
Debt Financing             21
Series F                   17
Private Equity Round       12
Grant                      12
Funding Round              10
Corporate Round             7
Convertible Note            6
Angel Round                 5
Equity Crowdfunding         5
Series G                    4
Non Equity Assistance       3
Series H                    2
Initial Coin Offering       2
Product Crowdfunding        2
Series D                    1
Pre Seed Round              1
Series I                    1
Series J                    1
Series B                    1
Name: second_latest_funding_type_column, dtype: int64

#### Extract all the latest years when each company was last funded.

In this section, we extract the latest and second latest years when each company was last funded. We do this by extracting the years information, from two columns, i.e. 'latest_funding_date' and 'second_latest_funding_date'. This information is important as we shall use it in the classification machine learning problem, in order to help identify all the companies that have recently received funds (recency > 2007)

In [27]:
# Extract the year substring from the 'second_latest_funding_date' column in the crunchbase dataframe

# first, convert both the 'latest_funding_date' and 'second_latest_funding_date' columns to the datetime format
df_crunchbase['latest_funding_date'] = pd.to_datetime(df_crunchbase['latest_funding_date'])
df_crunchbase['second_latest_funding_date'] = pd.to_datetime(df_crunchbase['second_latest_funding_date'])

# extract the years from both columns
df_crunchbase['latest_funding_year'] = df_crunchbase['latest_funding_date'].dt.year
df_crunchbase['second_latest_funding_year'] = df_crunchbase['second_latest_funding_date'].dt.year

# view the new datetime format in the two columns
df_crunchbase.head(3)

,name,operating_status,ipo,company_type,latest_funding_type,latest_funding_date,second_latest_funding_type,second_latest_funding_date,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year
0,Argo AI,Active,Private,For Profit,Corporate Round - Argo AI,2019-07-12,Funding Round - Argo AI,2017-02-13,46,7,NaN,Corporate Round,Funding Round,2019.0,2017.0
1,DeepAI,Seed,Private,NaN,Seed Round - DeepAI,2019-03-22,NaN,NaT,NaN,NaN,NaN,Seed Round,nan,2019.0,NaN
3,Juni Learning,Early Stage Venture,Private,For Profit,Series A - Juni Learning,2020-03-12,Seed Round - Juni Learning,2019-05-31,NaN,NaN,NaN,Series A,Seed Round,2020.0,2019.0


#### Drop extraneous columns from the crunchbase dataframe.

In this section, we drop a number of columns ["company_type","latest_funding_type","second_latest_funding_type","latest_funding_date","second_latest_funding_date"], because we have already extracted the necessary bits of information from each one of these columns. 

*   We have extracted the 'for-profit' companies from the 'company-type' column;
*   The latest investment types from the "latest_funding_type","second_latest_funding_type" columns; 
*   The latest funding years from the "latest_funding_date" and"second_latest_funding_date" columns.

In [28]:
# dropping the extraneous columns
df_crunchbase.drop(columns=["company_type","latest_funding_type","second_latest_funding_type","latest_funding_date","second_latest_funding_date"], inplace=True)

#### View the final df_crunchbase dataframe after dropping the five extraneous columns.

In [30]:
df_crunchbase.head(3)

,name,operating_status,ipo,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year
0,Argo AI,Active,Private,46,7,NaN,Corporate Round,Funding Round,2019.0,2017.0
1,DeepAI,Seed,Private,NaN,NaN,NaN,Seed Round,nan,2019.0,NaN
3,Juni Learning,Early Stage Venture,Private,NaN,NaN,NaN,Series A,Seed Round,2020.0,2019.0


#### Creating a separate dataframe for the name and founding year from the Viziologi dataset.

In [52]:
#Create a separate dataframe for the founding year and name from the Viziologi dataset
df_founded_subset = df_viziologi[["name",'founded']]
df_founded_subset.head(3)

,name,founded
0,Argo AI,2017
1,Call 9,2017
2,DeepAI,2017


#### Merge the founding year of each firm into the crunchbase dataset.



In [53]:
# merge the founding year of each firm into the crunchbase dataset
df_crunchbase_2=pd.merge(df_crunchbase,df_founded_subset, how='left', on='name')

# drop instance if the company was founded earlier than 2007
df_crunchbase_2.drop(df_crunchbase_2[df_crunchbase_2["founded"]<=2006].index, inplace=True)
df_crunchbase_2.reset_index(drop=True,inplace=True)

# view the first 3 rows of the df_crunchbase_2 dataframe
df_crunchbase_2.head(3)

,name,operating_status,ipo,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year,founded
0,Argo AI,Active,Private,46,7,NaN,Corporate Round,Funding Round,2019.0,2017.0,2017.0
1,DeepAI,Seed,Private,NaN,NaN,NaN,Seed Round,nan,2019.0,NaN,2017.0
2,Juni Learning,Early Stage Venture,Private,NaN,NaN,NaN,Series A,Seed Round,2020.0,2019.0,2017.0


#### Create success metrics from the df_crunchbase_2 dataframe.

In [54]:
#IPO and founded no earlier than 2007
success = df_crunchbase_2[(df_crunchbase_2["operating_status"]=="IPO") & (df_crunchbase_2['founded']>=2007)]
success=success.append(df_crunchbase_2[(df_crunchbase_2["ipo"]=="Public") & (df_crunchbase_2['founded']>=2007)])

#M&A and founded no earlier than 2010
success=success.append(df_crunchbase_2[(df_crunchbase_2["operating_status"]=="M&A") & (df_crunchbase_2['founded']>=2010)])

#Raised more than six rounds of funding (Series F,G,H,I,J)
for i in ["Series F","Series G", "Series H","Series I","Series J"]:
    success=success.append(df_crunchbase_2[df_crunchbase_2["latest_funding_type_column"]==i]["latest_funding_type_column"])
    
for i in ["Series F","Series G", "Series H","Series I","Series J"]:
    success=success.append(df_crunchbase_2[df_crunchbase_2["second_latest_funding_type_column"]==i]["second_latest_funding_type_column"])

#Raised Series A within 5 years from founding year
success=success.append(df_crunchbase_2[(df_crunchbase_2["latest_funding_type_column"]=="Series A") & (df_crunchbase_2['latest_funding_year']-df_crunchbase_2['founded']<=5)])
success=success.append(df_crunchbase_2[(df_crunchbase_2["second_latest_funding_type_column"]=="Series A") & (df_crunchbase_2['second_latest_funding_year']-df_crunchbase_2['founded']<=5)])

#Raised Series B within 6 years from founding year
success=success.append(df_crunchbase_2[(df_crunchbase_2["latest_funding_type_column"]=="Series B") & (df_crunchbase_2['latest_funding_year']-df_crunchbase_2['founded']<=6)])
success=success.append(df_crunchbase_2[(df_crunchbase_2["second_latest_funding_type_column"]=="Series B") & (df_crunchbase_2['second_latest_funding_year']-df_crunchbase_2['founded']<=6)])

#Raised Series C within 8 years from founding year
success=success.append(df_crunchbase_2[(df_crunchbase_2["latest_funding_type_column"]=="Series C") & (df_crunchbase_2['latest_funding_year']-df_crunchbase_2['founded']<=8)])
success=success.append(df_crunchbase_2[(df_crunchbase_2["second_latest_funding_type_column"]=="Series C") & (df_crunchbase_2['second_latest_funding_year']-df_crunchbase_2['founded']<=8)])

#Raised Series D within 10 years from founding year
success=success.append(df_crunchbase_2[(df_crunchbase_2["latest_funding_type_column"]=="Series D") & (df_crunchbase_2['latest_funding_year']-df_crunchbase_2['founded']<=10)])
success=success.append(df_crunchbase_2[(df_crunchbase_2["second_latest_funding_type_column"]=="Series D") & (df_crunchbase_2['second_latest_funding_year']-df_crunchbase_2['founded']<=10)])

#Raised Series E within 11 years from founding year
success=success.append(df_crunchbase_2[(df_crunchbase_2["latest_funding_type_column"]=="Series E") & (df_crunchbase_2['latest_funding_year']-df_crunchbase_2['founded']<=11)])
success=success.append(df_crunchbase_2[(df_crunchbase_2["second_latest_funding_type_column"]=="Series E") & (df_crunchbase_2['second_latest_funding_year']-df_crunchbase_2['founded']<=11)])

#Unicorns and Exited Unicorns
success=success.append(df_crunchbase_2[(df_crunchbase_2["hub"]=="Unicorn") | (df_crunchbase_2['hub']=='Exited Unicorn')])

# view the first 3 rows the new dataframe with the success metrics
success.head(3)

,name,operating_status,ipo,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year,founded
22,Alphabet,IPO,Public,"35,710","1,310",NaN,nan,nan,NaN,NaN,2015.0
69,NIO,IPO,Public,168,35,Exited Unicorn,Post,Post,2020.0,2020.0,2014.0
101,Casper,IPO,Public,12,33,Exited Unicorn,Series D,Series C,2019.0,2017.0,2013.0


#### Extract unicorn companies from the 'hub' column in the df_crunchbase_2 dataframe.

Here, we extract all companies that are unicorns from the 'hub' column, by encoding them as 1. For all the other companies that are not unicorns, they are encoded as 0. This information is important as we will use it in our classification machine learning models to identify whether startups are successful when they are unicorns or not.

In [56]:
# creating a new column to store the unicorn company labels 
df_crunchbase_2['unicorn']=np.nan

# extracting the unicorn companies from the 'hub' column 
df_crunchbase_2['hub'].fillna(0, inplace=True)
for i in range(len(df_crunchbase_2)):
    if 'Unicorn' in str(df_crunchbase_2['hub'][i]):
        df_crunchbase_2['unicorn'][i]=1
    else:
        df_crunchbase_2['unicorn'][i]=0

# viewing the first 3 rows of the new dataframe
df_crunchbase_2.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,operating_status,ipo,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year,founded,unicorn
0,Argo AI,Active,Private,46,7,0,Corporate Round,Funding Round,2019.0,2017.0,2017.0,0.0
1,DeepAI,Seed,Private,NaN,NaN,0,Seed Round,nan,2019.0,NaN,2017.0,0.0
2,Juni Learning,Early Stage Venture,Private,NaN,NaN,0,Series A,Seed Round,2020.0,2019.0,2017.0,0.0


#### Work out the unique values in the new 'unicorn' column, in the df_crunchbase_2 dataframe.

As can be observed from the results given below, there are 59 unicorn companies (unicorns were encoded as 1) and 524 other companies that are not unicorns (encoded as 0).

In [57]:
df_crunchbase_2['unicorn'].value_counts()

0.0    524
1.0     59
Name: unicorn, dtype: int64

#### Extract information about each company's intellectual property variable.

Here, we indicate the presence of an ip if a company currently has a patent and a trademark. If a company has both a patent and a trademark, then it is recorded as having an ip (dummy variable 1), and vice versa (dummy variable 0). This information is important as we will use it in our classification machine learning models to identify whether startups that have registered ip's are more successful than those which dont. 

In [58]:
# creating a new column to store the companies' 'ip' labels
df_crunchbase_2['ip']=np.nan

# removing the missing values in the patents and trademarks by replacing them with zeroes
df_crunchbase_2['patents'].fillna(0, inplace=True)
df_crunchbase_2['trademarks'].fillna(0, inplace=True)

# extracting the ip labels from the patents and trademarks columns and adding them to a new column
for i in range(len(df_crunchbase_2)):
    if df_crunchbase_2['patents'][i]!=0:
        df_crunchbase_2['ip'][i]=1
    elif df_crunchbase_2['trademarks'][i]!=0:
        df_crunchbase_2['ip'][i]=1
    else:
        df_crunchbase_2['ip'][i]=0

# viewing the first 3 rows of the new dataframe
df_crunchbase_2.head(3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,name,operating_status,ipo,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year,founded,unicorn,ip
0,Argo AI,Active,Private,46,7,0,Corporate Round,Funding Round,2019.0,2017.0,2017.0,0.0,1.0
1,DeepAI,Seed,Private,0,0,0,Seed Round,nan,2019.0,NaN,2017.0,0.0,0.0
2,Juni Learning,Early Stage Venture,Private,0,0,0,Series A,Seed Round,2020.0,2019.0,2017.0,0.0,0.0


#### Feature engineering the final target variable (success).

Here, we create/ feature-engineer a new variable, labelled as 'success'. This new variable will serve as our final target/ response variable in our classification machine learning model.

To create this target variable, we encode all those companies that appear in the 'success' datafraeme as 1, and 0 for all the other companies that do not appear in the 'success' dataframe.

In [60]:
# creating a new column to store the new success labels
df_crunchbase_2["success"]=np.nan

# encoding the companies that appear in the 'success' dataframe as 1 and the rest as 0 
for i in range(len(df_crunchbase_2)):
    if i in range(len(success)):
        df_crunchbase_2["success"][i]=1
    else:
        df_crunchbase_2["success"][i]=0
        
# viewing the first 3 rows of the new dataframe        
df_crunchbase_2.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,name,operating_status,ipo,patents,trademarks,hub,latest_funding_type_column,second_latest_funding_type_column,latest_funding_year,second_latest_funding_year,founded,unicorn,ip,success
0,Argo AI,Active,Private,46,7,0,Corporate Round,Funding Round,2019.0,2017.0,2017.0,0.0,1.0,1.0
1,DeepAI,Seed,Private,0,0,0,Seed Round,nan,2019.0,NaN,2017.0,0.0,0.0,1.0
2,Juni Learning,Early Stage Venture,Private,0,0,0,Series A,Seed Round,2020.0,2019.0,2017.0,0.0,0.0,1.0


16. Work out the unique values in the new 'success' column, in the df_crunchbase_2 dataframe

As can be seen from the results given below, there are 164 successful companies (successful companies were encoded as 1) and the rest 394 companies that are not successful (encoded as 0)

In [63]:
df_crunchbase_2['success'].value_counts()

0.0    419
1.0    164
Name: success, dtype: int64

## FINAL MERGING OF THE TWO DATAFRAMES (VIZIOLOGI AND CRUNCHBASE) INTO A SINGLE DATAFRAME (DF_MERGED)

#### Merge the df_viziologi and df_crunchbase dataframes into one single dataframe (df_merged).

In [62]:
#Merge Viziologi dataset with Crunchbase dataset
df_merged=pd.merge(df_viziologi,df_crunchbase_2,how='right', on=['name'] )

#Drop rows that have NaN as the firm name
df_merged.drop(df_merged[df_merged["name"]=='nan'].index, inplace=True)
df_merged.reset_index(drop=True, inplace=True)

#Drop rows that exist in the Crunchbase dataset but have no corresponding instance in the Viziologi dataset
df_merged.drop(df_merged[459:489].index, inplace=True)
df_merged.reset_index(drop=True, inplace=True)

#Rename columns
df_merged.drop(columns=['founded_y','hub'], inplace=True)
df_merged.rename(columns={'founded_x':'founded'}, inplace=True)
df_merged.rename(columns={'hq':'city'}, inplace=True)
df_merged.rename(columns={'area':'state'}, inplace=True)

#### Print out the names of all the columns in the merged dataframe.

In [65]:
# print out the columns present in the df_merged dataset
df_merged.columns

Index(['name', 'sector', 'category', 'business_patterns', 'description',
       'city', 'state', 'country', 'founded', 'tags', 'operating_status',
       'ipo', 'patents', 'trademarks', 'latest_funding_type_column',
       'second_latest_funding_type_column', 'latest_funding_year',
       'second_latest_funding_year', 'unicorn', 'ip', 'success'],
      dtype='object')

#### Write out the newly merged dataframe (df_merged) to an external csv file.

In [67]:
df_merged.to_csv('df_merged.csv', index=False)